In [6]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import pickle
import os
import json
from config import config
import contractions
import pickle
import mysql.connector
from mysql.connector import Error
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_classif, RFE
import statsmodels.api as sm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score,f1_score
from sklearn.metrics import auc, average_precision_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from string import punctuation
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("white")
plt.rcParams['figure.figsize'] = (18, 5)
import warnings
warnings.simplefilter('ignore', FutureWarning)
warnings.simplefilter('ignore', UserWarning)

In [7]:
class ToPandasDF():
    def __init__(self, password, host, database, user):

        self.password = password
        self.host = host
        self.database = database
        self.user = user
        
    
    def MySQLconnect(self, query):
        
        try:
            connection = mysql.connector.connect(host=self.host, 
                                                 database=self.database, 
                                                 password=self.password,
                                                 user=self.user)

            if connection.is_connected():

                print("Successfully connected to the database")

                cursor = connection.cursor()
                query = query
                cursor.execute(query)

                data = cursor.fetchall()

                df = pd.DataFrame(data, columns = ['date', 'tweet'])
        except Error as e:
            print(e)
            
        cursor.close()
        connection.close()
        
        return df

if __name__ == '__main__':
    
    t = ToPandasDF(config.PASSWORD, config.HOST, config.DATABASE, config.USER)
    data = t.MySQLconnect("SELECT created_at, tweet FROM `twitterdb`.`twitter_table`;")

Successfully connected to the database


In [8]:
print(data.shape)
data.head(10)

(690, 2)


,date,tweet
0,2022-07-05 09:28:44,RT @JaneBrownNews: Good morning #Toronto! Toda...
1,2022-07-05 04:47:40,"RT @MCSCanada: 15 yr old, John Morris #missing..."
2,2022-07-05 04:57:57,#VladimirPutin:The Rise of a Villain https://t...
3,2022-07-05 09:28:44,RT @JaneBrownNews: Good morning #Toronto! Toda...
4,2022-07-05 04:47:40,"RT @MCSCanada: 15 yr old, John Morris #missing..."
5,2022-07-05 04:57:57,#VladimirPutin:The Rise of a Villain https://t...
6,2022-07-05 01:06:01,The Argos are trailing the Blue Bombers 17-3 a...
7,2022-07-05 02:09:26,RT @hiodollz: Spent the whole weekend at the s...
8,2022-07-05 05:07:44,RT @KarmSumal: The Grizzlies definitely deserv...
9,2022-07-02 08:49:03,RT @layzhang: love what you do and you will al...


In [9]:
print(data.isnull().sum())
# These are duplicated rows
data[data.duplicated()].head(30)

date     6
tweet    0
dtype: int64


,date,tweet
3,2022-07-05 09:28:44,RT @JaneBrownNews: Good morning #Toronto! Toda...
4,2022-07-05 04:47:40,"RT @MCSCanada: 15 yr old, John Morris #missing..."
5,2022-07-05 04:57:57,#VladimirPutin:The Rise of a Villain https://t...
23,2022-07-05 04:47:40,"RT @MCSCanada: 15 yr old, John Morris #missing..."
25,2022-07-05 01:06:01,The Argos are trailing the Blue Bombers 17-3 a...
27,2022-07-05 05:07:44,RT @KarmSumal: The Grizzlies definitely deserv...
28,2022-07-02 08:49:03,RT @layzhang: love what you do and you will al...
29,2022-06-28 18:01:14,There are some exciting changes happening over...
30,2022-06-28 16:43:23,#Chronic #Pain #Relief #Requires #High #Levels...
31,2022-07-02 14:00:05,#SaturdayMotivation She’s #Highly 👉🏽 #Consiste...


In [10]:
# Data with no duplicate
data=data[~data.duplicated()]
print(data.shape)
data.head(30)

(360, 2)


,date,tweet
0,2022-07-05 09:28:44,RT @JaneBrownNews: Good morning #Toronto! Toda...
1,2022-07-05 04:47:40,"RT @MCSCanada: 15 yr old, John Morris #missing..."
2,2022-07-05 04:57:57,#VladimirPutin:The Rise of a Villain https://t...
6,2022-07-05 01:06:01,The Argos are trailing the Blue Bombers 17-3 a...
7,2022-07-05 02:09:26,RT @hiodollz: Spent the whole weekend at the s...
8,2022-07-05 05:07:44,RT @KarmSumal: The Grizzlies definitely deserv...
9,2022-07-02 08:49:03,RT @layzhang: love what you do and you will al...
10,2022-06-28 18:01:14,There are some exciting changes happening over...
11,2022-06-28 16:43:23,#Chronic #Pain #Relief #Requires #High #Levels...
12,2022-07-02 14:00:05,#SaturdayMotivation She’s #Highly 👉🏽 #Consiste...
